#Upload dos arquivos

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
%%bash
# Copiando e colando os arquivos da análise do Google Drive para esse ambiente de trabalho.
cp /content/drive/MyDrive/Colab\ Notebooks/Atividade_Complementar_Bioinf/Chr7-reference.fasta.gz .
cp /content/drive/MyDrive/Colab\ Notebooks/Atividade_Complementar_Bioinf/caso3-tumor-pulmonar_R1.fastq.gz .
cp /content/drive/MyDrive/Colab\ Notebooks/Atividade_Complementar_Bioinf/caso3-tumor-pulmonar_R2.fastq.gz .

# Configuração de ambiente padrão


In [ ]:
%%bash
# Instalar miniconda e mamba
wget --quiet https://github.com/conda-forge/miniforge/releases/latest/download/Miniforge3-Linux-x86_64.sh
bash Miniforge3-Linux-x86_64.sh -b -p /usr/local -u
mamba init
mamba config set auto_activate_base true
mamba shell init --shell bash --root-prefix=~/.local/share/mamba
rm Miniforge3-Linux-x86_64.sh

In [ ]:
%%bash
# Instalar os software que serão usados
mamba install -c bioconda -c conda-forge fastqc bwa gatk4 cutadapt samtools bedtools ensembl-vep=115 -y

In [ ]:
%%bash
# Criar as pastas para organizar os arquivos que serão gerados
mkdir -p raw fastqc cutadapt bwa gatk mutect vep reference

In [ ]:
%%bash
# Mover arquivos para as suas respectivas pastas e descompactar fasta.gz
mv *.fastq.gz raw/
gunzip -c Chr7-reference.fasta.gz > reference/Chr7-reference.fasta
rm -rf Chr7-reference.fasta.gz sample_data/

# Controle de Qualidade

In [ ]:
%%bash
# Gerar o relatório da qualidade dos reads de sequenciamento para os pacientes 1 e 2
fastqc -o fastqc/ raw/caso3-tumor-pulmonar_R1.fastq.gz raw/caso3-tumor-pulmonar_R2.fastq.gz

In [ ]:
%%bash
# Filtrar sequências muito pequenas e cortar as pontas
cutadapt \
  -u 5 -U 5 \
  -u -5 -U -5 \
  -m 90 \
  -o cutadapt/caso3-tumor-pulmonar-trimmed_R1.fastq.gz \
  -p cutadapt/caso3-tumor-pulmonar-trimmed_R2.fastq.gz \
  raw/caso3-tumor-pulmonar_R1.fastq.gz \
  raw/caso3-tumor-pulmonar_R2.fastq.gz

# Mapeamento com a referência do genoma humano

In [ ]:
%%bash
# Indexar o genoma humano de referência para realizar o mapeamento e posterior remoção de contaminante
samtools faidx reference/Chr7-reference.fasta
bwa index reference/Chr7-reference.fasta

In [ ]:
%%bash
# Proceder com o alinhamento contra o genoma humano
bwa mem reference/Chr7-reference.fasta \
  -R '@RG\tID:sample1\tSM:sample1\tPL:ILLUMINA\tLB:lib1\tPU:unit1' \
  cutadapt/caso3-tumor-pulmonar-trimmed_R1.fastq.gz \
  cutadapt/caso3-tumor-pulmonar-trimmed_R2.fastq.gz \
  | samtools sort -o bwa/caso3-tumor-pulmonar-mapped.bam

In [ ]:
%%bash
# Indexar o bam para gerar o bai e visualizar no IGV
samtools index bwa/caso3-tumor-pulmonar-mapped.bam

# Chamada de Variantes

In [ ]:
%%bash
# Indexar a referência e BAM para chamada de variantes com GATK
samtools dict reference/Chr7-reference.fasta > reference/Chr7-reference.dict
samtools index bwa/caso3-tumor-pulmonar-mapped.bam

In [ ]:
%%bash
# Chamada de variantes com mutect2
 gatk Mutect2 -R reference/Chr7-reference.fasta \
    -I bwa/caso3-tumor-pulmonar-mapped.bam \
    -O mutect/caso3-tumor-pulmonar-unfiltered.vcf

In [ ]:
%%bash
# Filtragem de variantes
gatk FilterMutectCalls \
    -R reference/Chr7-reference.fasta \
    -V mutect/caso3-tumor-pulmonar-unfiltered.vcf \
    -O mutect/caso3-tumor-pulmonar-filtered.vcf

# Anotação de Variantes

In [ ]:
%%bash
# Script oficial baixa cache do genoma humano
vep_install -a cf -s homo_sapiens -y GRCh38 --NO_UPDATE

In [ ]:
%%bash
# Index VCF files para anotação
bgzip mutect/caso3-tumor-pulmonar-filtered.vcf
tabix -f -p vcf mutect/caso3-tumor-pulmonar-filtered.vcf.gz


In [ ]:
%%bash
# Anotação de variantes a nível de proteína
vep -i mutect/caso3-tumor-pulmonar-filtered.vcf.gz \
    --output_file vep/caso3-tumor-pulmonar-filtered.vep.tsv \
    --tab --force_overwrite \
    --offline --cache --dir_cache ~/.vep \
    --assembly GRCh38 \
    --symbol --hgvs --canonical --af_1kg --max_af \
    --everything --pick\
    --stats_file vep/caso3-tumor-pulmonar-filtered.vep.html


# Interpretação dos resultados

O paciente apresentou duas importantes alterações genéticas que guiam o tratamento: a mutação L858R no gene EGFR e a mutação V600E no gene BRAF. O gene EGFR codifica o Receptor do Fator de Crescimento Epidérmico e a variante identificada, que é uma substituição de Leucina (L) por Arginina (R) no códon 858, resulta em um ganho de função na atividade de tirosina quinase. Essa hiperativação causa um aumento na sinalização celular contínua, estimulando a proliferação e sobrevivência celular, e está fortemente associada ao adenocarcinoma de pulmão de não pequenas células (NSCLC). Para o manejo dessa alteração, o tratamento é tipicamente focado no uso de inibidores de tirosina quinase (TKIs), mas há também a opção terapêutica inovadora do Datopotamab Deruxtecan, um anticorpo monoclonal conjugado que entrega um agente citotóxico diretamente ao tumor.

A segunda variante, V600E no gene BRAF, é crítica por se situar em um gene fundamental da via de sinalização MAPK/ERK, que controla o crescimento celular. Esta mutação representa a substituição de Valina (V) por Glutamato (E) no códon 600, resultando na ativação constitutiva (permanente) da via MAPK, o que leva à proliferação celular incontrolada. Essa alteração está associada a diversos tipos de câncer, como melanoma, tireoide, colorretal e pulmonar, e seu tratamento é tipicamente realizado através de regimes de terapias-alvo combinadas, como a associação de Dabrafenibe com Trametinibe, ou Encorafenibe com Binimetinibe.